In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Load Preprocessed Data

In [5]:
face_all = np.load("npy/face_all.npy", allow_pickle = True)
nirs_all= np.load("npy/nirs_all.npy", allow_pickle = True)
clip_idx = np.load("npy/clip_idx.npy", allow_pickle = True)

## S2S-RNN-Autoencoder (Unimodal Embeddings)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
class S2SAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(S2SAutoencoder, self).__init__()
        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, input_size, batch_first=True)

    def forward(self, x):
        encoder_out, (hidden, cell) = self.encoder(x)
        encoder_last_timestep = encoder_out[:, -1, :] # Get the LSTM output at the last time step
        decoder_out, _ = self.decoder(encoder_out, (hidden, cell))
        
        return decoder_out, encoder_last_timestep

In [8]:
face_reshaped = face_all
nirs_reshaped = nirs_all

for reshaped in [face_reshaped,nirs_reshaped]:
    unimodal_autoencoder = S2SAutoencoder(reshaped.shape[2], reshaped.shape[2]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(unimodal_autoencoder.parameters(), lr=0.001)
    batch_size = 128

    uni_tensor = torch.from_numpy(reshaped).float().to(device)
    label_tensor = torch.tensor(clip_idx).to(device)
    train_dataset = TensorDataset(uni_tensor, label_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    

    num_epochs = 20
    all_bottlenecks = []
    ys = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_data, y in train_loader:
            optimizer.zero_grad()
            outputs, bottleneck = unimodal_autoencoder(batch_data)
            loss = criterion(outputs, batch_data)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
            # Append the bottleneck embeddings to the list
            if epoch == num_epochs - 1:
                all_bottlenecks.append(bottleneck)
                ys.append(y)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')
        
    torch.save(unimodal_autoencoder.state_dict(), 'models/feature_selection/unimodal_autoencoder.pth')
    all_bottlenecks_combined = torch.cat(all_bottlenecks, dim=0)
    ys = torch.cat(ys, dim=0)
    
    cluster_idx = []
    subject_label = []
    for y in ys:
        cluster_idx.append(int(str(y.item())[:-4]+str(0)))
        subject_label.append((int(str(y.item())[:-3])))
    
    embeddings = all_bottlenecks_combined.cpu()
    embeddings_np = embeddings.detach().numpy()
    
    unimodal_df = pd.DataFrame(data=embeddings_np, columns=[f"dim_{i+1}" for i in range(embeddings_np.shape[1])])
    unimodal_df['idx'] = ys.cpu()
    unimodal_df['cluster_idx'] = cluster_idx
    unimodal_df['subject_idx'] = subject_label
    unimodal_df['t'] = unimodal_df['idx'].apply(lambda x: str(x)[-3:])

    unimodal_df = unimodal_df.sort_values(by=['subject_idx', 't'])
    unimodal_df = unimodal_df.reset_index(drop=True)
    unimodal_df.to_csv('output/feature_selection/unimodal_embeddings_dim'+str(reshaped.shape[2])+'.csv')
    
    print('unimodal_embeddings_dim'+str(reshaped.shape[2])+'.csv saved!')

Epoch [1/20], Loss: 0.1552486359750675
Epoch [2/20], Loss: 0.10694790485745204
Epoch [3/20], Loss: 0.10246796314784186
Epoch [4/20], Loss: 0.10080909536606505
Epoch [5/20], Loss: 0.10003180716008032
Epoch [6/20], Loss: 0.09956258511904939
Epoch [7/20], Loss: 0.09922375103155623
Epoch [8/20], Loss: 0.09897980302123674
Epoch [9/20], Loss: 0.09879538746286089
Epoch [10/20], Loss: 0.09861416492684764
Epoch [11/20], Loss: 0.09845144467766512
Epoch [12/20], Loss: 0.0983365008560196
Epoch [13/20], Loss: 0.09826319518578804
Epoch [14/20], Loss: 0.09816904080749567
Epoch [15/20], Loss: 0.09814021623538333
Epoch [16/20], Loss: 0.09806938392310802
Epoch [17/20], Loss: 0.09807119976907884
Epoch [18/20], Loss: 0.09802876245353094
Epoch [19/20], Loss: 0.09799605605883746
Epoch [20/20], Loss: 0.09795771288288616
embeddings_dim35.csv saved!
Epoch [1/20], Loss: 0.3670540795445106
Epoch [2/20], Loss: 0.18490833247313496
Epoch [3/20], Loss: 0.16829946970782633
Epoch [4/20], Loss: 0.16200265637401803
Epoc

## FCN-Autoencoder (Multimodal Integration)

In [9]:
face_df=pd.read_csv('output/feature_selection/unimodal_embeddings_dim35.csv')
fnirs_df=pd.read_csv('output/feature_selection/unimodal_embeddings_dim40.csv')

multimodal = pd.concat([fnirs_df.iloc[:, 1:-4], face_df.iloc[:, 1:-4]], axis = 1)
multimodal_all = multimodal.values
multimodal_idx = fnirs_df['idx'].values

In [10]:
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(75, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 75),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

In [11]:
batch_size = 128

multimodal_tensor = torch.from_numpy(multimodal_all).float().to(device)
label_tensor = torch.tensor(multimodal_idx).to(device)
train_dataset = TensorDataset(multimodal_tensor, label_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

autoencoder = Autoencoder(encoding_dim=75).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

num_epochs = 20
all_bottlenecks = []
ys = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_data, y in train_loader:
        optimizer.zero_grad()
        outputs, bottleneck = autoencoder(batch_data)  # Access the data from the DataLoader
        loss = criterion(outputs, batch_data)  # Compare with the input data
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Append the bottleneck embeddings to the list
        if epoch == num_epochs - 1:
            all_bottlenecks.append(bottleneck)
            ys.append(y)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')
    
torch.save(autoencoder.state_dict(), 'models/feature_selection/multimodal_autoencoder_dim75.pth')
all_bottlenecks_combined = torch.cat(all_bottlenecks, dim=0)
ys = torch.cat(ys, dim=0)

cluster_idx = []
subject_label = []
for y in ys:
    cluster_idx.append(int(str(y.item())[:-4]+str(0)))
    subject_label.append((int(str(y.item())[:-3])))
    
embeddings = all_bottlenecks_combined.cpu()
embeddings_np = embeddings.detach().numpy()

multimodal_df = pd.DataFrame(data=embeddings_np, columns=[f"dim_{i+1}" for i in range(embeddings_np.shape[1])])
multimodal_df['idx'] = ys.cpu()
multimodal_df['cluster_idx'] = cluster_idx
multimodal_df['subject_idx'] = subject_label
multimodal_df['t'] = multimodal_df['idx'].apply(lambda x: str(x)[-3:])

multimodal_df = multimodal_df.sort_values(by=['subject_idx', 't'])
multimodal_df = multimodal_df.reset_index(drop=True)
multimodal_df.to_csv('output/feature_selection/multimodal_embeddings_dim75.csv')

Epoch [1/20], Loss: 0.05494650969542878
Epoch [2/20], Loss: 0.035834317366300834
Epoch [3/20], Loss: 0.03289301559567339
Epoch [4/20], Loss: 0.03218977303819789
Epoch [5/20], Loss: 0.03209191805710236
Epoch [6/20], Loss: 0.0320796100291321
Epoch [7/20], Loss: 0.032078312846603306
Epoch [8/20], Loss: 0.03207520301246127
Epoch [9/20], Loss: 0.03207374452801661
Epoch [10/20], Loss: 0.03207119166858053
Epoch [11/20], Loss: 0.032068715026901784
Epoch [12/20], Loss: 0.03206831160214355
Epoch [13/20], Loss: 0.03206578362470288
Epoch [14/20], Loss: 0.032064745819453684
Epoch [15/20], Loss: 0.032063153802801836
Epoch [16/20], Loss: 0.032059678576557234
Epoch [17/20], Loss: 0.03205494557070149
Epoch [18/20], Loss: 0.03204210499618711
Epoch [19/20], Loss: 0.03202885965293629
Epoch [20/20], Loss: 0.03202324041173297
